In [1]:
print('Importing')
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
from datetime import datetime 
print('Done importing')

Importing
Done importing


In [2]:
start_time = datetime.now()
print(start_time)

2024-06-03 20:05:52.914486


In [3]:
pathologies = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
               'Lung Opacity', 'Pneumonia', 'Pleural Effusion', 'Pleural Other',
               'Fracture', 'Support Devices']
hpc = False
mode = 1
print(sys.argv)
if (len(sys.argv) > 1 and sys.argv[1] == 'hpc'):
    hpc = True
    if (len(sys.argv) > 2):
        mode = int(sys.argv[2])

pathology = pathologies[mode]
print('pathology:', pathology)

['/Users/wilsonduan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py', '--f=/Users/wilsonduan/Library/Jupyter/runtime/kernel-v2-873q3mzvS6N7xqE.json']
pathology: Enlarged Cardiomediastinum


In [4]:
lr = 0.0002
n_epochs = 15
n_cpu = 4 if hpc else 0
batch_size = 32
img_size = 256
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print(hpc, device, n_epochs, n_cpu, img_size)

False cpu 15 0 256


In [5]:
if (hpc):
    labels_path_train = '/groups/CS156b/data/student_labels/train2023.csv'
    labels_path_test = '/groups/CS156b/data/student_labels/solution_ids.csv'
    img_dir = '/groups/CS156b/data'

    df_train = pd.read_csv(labels_path_train)[:-1]
else:
    labels_path_train = '../data/train/labels/labels.csv'
    labels_path_test = '../data/test/ids.csv'
    img_dir = '../data'

    df_train = pd.read_csv(labels_path_train)

df_train = df_train[['Path', pathology]]
df_train = df_train.dropna()
df_test = pd.read_csv(labels_path_test)
print(df_train.head())
print(df_test.head())

                                        Path  Enlarged Cardiomediastinum
5    train/pid17532/study1/view1_frontal.jpg                        -1.0
6    train/pid17532/study1/view2_lateral.jpg                        -1.0
7    train/pid05208/study1/view1_frontal.jpg                        -1.0
9    train/pid35409/study2/view1_frontal.jpg                         1.0
14  train/pid18261/study13/view1_frontal.jpg                        -1.0
   Unnamed: 0  Id                                    Path
0           0  18  test/pid56785/study1/view1_frontal.jpg
1           1  19  test/pid56785/study1/view2_lateral.jpg
2           2  44  test/pid57943/study1/view1_frontal.jpg
3           3  45  test/pid57943/study2/view1_frontal.jpg
4           4  57  test/pid54805/study1/view1_frontal.jpg


In [18]:
def parse_labels(df):
    df = df[['Path', pathology]]
    df = df.dropna()
    return df

class TrainImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        if (hpc):
            self.img_labels = parse_labels(pd.read_csv(annotations_file)[:-1])
        else:
            self.img_labels = parse_labels(pd.read_csv(annotations_file))
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path']
        img_path = os.path.join(self.img_dir, img_path)

        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        label_num = list(row)[-1] + 1 # -1 => 0, 0 => 1, 1 => 2
        label = torch.tensor(label_num).long()

        if self.transform:
            image = self.transform(image)

        return image, label
    
class TestImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path']
        img_path = os.path.join(self.img_dir, img_path)

        # image = read_image(img_path)
        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        label = row['Id']

        if self.transform:
            image = self.transform(image)

        return image, label

In [19]:
# transform with random flipping and cropping:
transform = transforms.Compose([
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop((256, 256)),
    # transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
])

training_data = TrainImageDataset(labels_path_train, img_dir, transform=transform)
test_data = TestImageDataset(labels_path_test, img_dir, transform=transform)

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=n_cpu)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [20]:
model = nn.Sequential(
    nn.Conv2d(3, 8, kernel_size=(3, 3)),
    nn.BatchNorm2d(num_features=8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),

    nn.Conv2d(8, 4, kernel_size=(3, 3)),
    nn.BatchNorm2d(num_features=4),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),

    nn.Flatten(),
    nn.Linear(62*62*4, 64),
    nn.ReLU(),
    nn.Linear(64, 3),
)
model = nn.DataParallel(model)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

In [21]:
# store metrics
training_loss_history = np.zeros(n_epochs)
validation_loss_history = np.zeros(n_epochs)
early_stop = False

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}:')
    # train
    model.train()
    for i, data in enumerate(train_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()

        # track training loss
        training_loss_history[epoch] += loss.item()

        # check if stop
        current_time = datetime.now()
        time_difference = current_time - start_time
        duration_in_s = time_difference.total_seconds() 
        hours = divmod(duration_in_s, 3600)[0]
        if (hours > 18):
            early_stop = True
            break
    
    training_loss_history[epoch] /= len(train_dataloader)
    print(f'Training Loss: {training_loss_history[epoch]:0.4f}')

    if (early_stop):
        break

Epoch 1/15:
Training Loss: 0.9511
Epoch 2/15:
Training Loss: 1.1237
Epoch 3/15:
Training Loss: 0.7677
Epoch 4/15:
Training Loss: 0.6914
Epoch 5/15:
Training Loss: 0.5821
Epoch 6/15:


KeyboardInterrupt: 

In [15]:
# get predictions on test set
rows_list = []
with torch.no_grad():
    model.eval()
    for i, data in enumerate(test_dataloader):
        images, ids = data
        images, ids = images.to(device), ids.to(device)
        
        output = model(images).cpu()
        output = nn.functional.softmax(output)
        output = (-1 * output[:, 0] + output[:, 2]) / 2
        for preds, id in zip(output, ids):
            rows_list.append([int(id)] + [float(preds)])

df_output = pd.DataFrame(rows_list, columns=['Id', pathology])
df_output.head()

/Users/wilsonduan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


,Id,Enlarged Cardiomediastinum
0,18,-0.019304
1,19,-0.098539
2,44,-0.084236
3,45,0.000610
4,57,-0.071353


In [ ]:
if (hpc):
    output_dir = '/groups/CS156b/2024/BroadBahnMi/predictions'
    model_dir = '/groups/CS156b/2024/BroadBahnMi/models'
else:
    output_dir = '../predictions'
    model_dir = '../models'

time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

filename = '_'.join(pathology.split()) + '_' + time
filename = filename.replace(' ', '_')
full_path = os.path.join(output_dir, f'cnn_preds_{filename}.csv')
df_output.to_csv(full_path, index=False)

model_path = os.path.join(model_dir, f'cnn_{filename}.pt')
torch.save(model.state_dict(), model_path)